In [2]:
%load_ext autoreload
%autoreload 2
import spacy
import scispacy
from scispacy.linking import EntityLinker
import en_core_sci_lg
from scify.nlp import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
text = "Spinal and bulbar muscular atrophy (SBMA) is an inherited motor neuron disease caused by the expansion of a polyglutamine tract within the androgen receptor (AR). SBMA can be caused by this easily."

In [3]:
from scispacy.abbreviation import AbbreviationDetector
from spacy.pipeline import merge_entities
from scispacy.linking import EntityLinker

nlp = load_sci_pipe()

In [5]:
#linker_umls = EntityLinker(resolve_abbreviations=True, name="umls")
#nlp.add_pipe(linker_umls)

In [10]:
from textacy.corpus import Corpus
from scify.consts import labels

for label in labels:
    nlp.vocab.strings.add(label)

corp = Corpus(nlp).load(nlp, "../data/experiments/pubmed_20n1015_annotated_756")

#https://github.com/explosion/spaCy/issues/860
#Doc.set_extension("annotated", default=None)
#doc = nlp(text)

In [328]:
#invalid pattern
construct_pattern('causes|nsubj|END_ENTITY causes|dobj|START_ENTITY')

[{'SPEC': {'NODE_NAME': 'causes'}, 'PATTERN': {'LEMMA': 'cause'}},
 {'SPEC': {'NODE_NAME': 'END_ENTITY',
   'NBOR_RELOP': '>',
   'NBOR_NAME': 'causes'},
  'PATTERN': {'DEP': 'nsubj', 'POS': 'NOUN'}},
 {'SPEC': {'NODE_NAME': 'START_ENTITY',
   'NBOR_RELOP': '>',
   'NBOR_NAME': 'causes'},
  'PATTERN': {'DEP': 'dobj', 'POS': 'NOUN'}}]